<a href="https://colab.research.google.com/github/khinan/folium_project_n3ts/blob/main/Khinanti_10_1_Benda_Baru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import folium
import pandas as pd
import geopandas as gpd
from google.colab import drive

In [60]:


drive.mount('/content/drive')

indonesia_admin = gpd.read_file('/content/drive/MyDrive/ADMIN_INDO/ADMINISTRASI_AR_DESAKEL_10K.shp')

aoi_admin = indonesia_admin[indonesia_admin['WADMKK'] == 'Kota Tangerang Selatan']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:

csv_path = '/content/drive/MyDrive/tangsel_kelurahan_KK.csv'


admin_gdf = aoi_admin


csv_data = pd.read_csv(csv_path, header = 2, delimiter= ';')

print(csv_data.head())
print(admin_gdf.head())

csv_data['Kelurahan'] = csv_data['Kelurahan'].str.lower().str.replace(' ', '')
admin_gdf['NAMOBJ'] = admin_gdf['NAMOBJ'].str.lower().str.replace(' ', '')

merged_data = pd.merge(admin_gdf, csv_data, left_on='NAMOBJ', right_on='Kelurahan', how='left')


merged_gdf = gpd.GeoDataFrame(merged_data, geometry=merged_data.geometry)

print(merged_gdf.head())


  Kecamatan        Kelurahan  Kepala Keluarga
0   Serpong           CIATER             8487
1   Serpong        RAWABUNTU             9028
2   Serpong   RAWAMEKAR JAYA             5784
3   Serpong  LENGKONG GUDANG             3855
4   Serpong   LENGKONG WETAN             3339
       OBJECTID        NAMOBJ       FCODE REMARK  \
24594   24595.0     bambuapus  BA03070040   None   
24595   24596.0     bendabaru  BA03070040   None   
24596   24597.0  cempakaputih  BA03070040   None   
24597   24598.0     cireundeu  BA03070040   None   
24598   24599.0       kedaung  BA03070040   None   

                                           METADATA     SRS_ID KDBBPS KDCBPS  \
24594  TASWIL1000020230928_DATA_BATAS_DESAKELURAHAN  SRGI 2013   None   None   
24595  TASWIL1000020230928_DATA_BATAS_DESAKELURAHAN  SRGI 2013   None   None   
24596  TASWIL1000020230928_DATA_BATAS_DESAKELURAHAN  SRGI 2013   None   None   
24597  TASWIL1000020230928_DATA_BATAS_DESAKELURAHAN  SRGI 2013   None   None   
24598  TASW

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [84]:
filtered_gdf = merged_gdf[merged_gdf['NAMOBJ'] == 'bendabaru']

m = folium.Map(location=[-6.2, 106.6], zoom_start=11)

choropleth = folium.Choropleth(
    geo_data=filtered_gdf,
    name='choropleth',
    data=filtered_gdf,
    columns=['NAMOBJ', 'Kepala Keluarga'],
    key_on= "feature.properties.NAMOBJ",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=1,
    legend_name="Jumlah KK"
).add_to(m)

jumlah_KK = folium.GeoJson(
    data=merged_gdf,
    name='Jumlah_KK',
    style_function=lambda feature: {'fillOpacity': 0, 'color': 'transparent'},
    popup = folium.GeoJsonPopup(
        fields=['NAMOBJ','13513'],
        aliases=['Kelurahan','Jumlah KK']
    )
).add_to(choropleth)

folium.LayerControl().add_to(m)